In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <list>
using namespace TEMPLET;
using namespace std;

In [ ]:
const int NUM_WORKERS = 5;
const int TASK_TIME = 10 * 1000; // ms
const int NUM_TASKS = 10;
double arr[NUM_TASKS];
int cur;

In [ ]:
struct worker;
list<worker*> idle_worker_list;

In [ ]:
struct worker:task {
    /**/
	int number;
	int pos_in_arr;
	/**/
	worker():task(){}
    /**/
    void put_number_and_pos(int n, int pos){
		number = n;
		pos_in_arr = pos;
		
        json in;
		in["name"] = "squared-number-task";
		in["inputs"]["n"] = n;
		in["inputs"]["time"] = TASK_TIME;
		input(in);
    }
    
    int get_squared_number(){
        int n=0;
        json out=result();
        n = out["squared-n"];
        return n;
    }
    /**/
    void on_worker_ready(){
        // put the result
        /**/arr[pos_in_arr] = get_squared_number();/**/
        idle_worker_list.push_back(this);
        run_tasks();
    }
    
    static void run_tasks(){
        // run tasks while having tasks and free workers
        while(!idle_worker_list.empty() && /**/cur < NUM_TASKS/**/){
            worker*w = idle_worker_list.front();
            idle_worker_list.pop_front();

            /**/w->put_number_and_pos(arr[cur],cur);cur++;/**/

            w->submit([=](){w->on_worker_ready();});
        }
    }
};

In [ ]:
{
    taskengine eng("your_everest_login", "your_everest_password");
    string token;
    eng.get_access_token(token);
    cout <<"access token: "<< token << endl;
    eng.get_app_description("your app's name");    
}

In [ ]:
taskengine eng("access token");

worker worker_arr[NUM_WORKERS];
    
for(int i=0;i<NUM_WORKERS;i++){
    worker_arr[i].set_app_id("your app' id");
    worker_arr[i].set_engine(eng);
    idle_worker_list.push_back(&worker_arr[i]);
}

/**/
for(int i=0;i<NUM_TASKS;i++){ arr[i] = i; cout << "arr[" << i << "]=" << arr[i] << endl;}
cur = 0;
/**/

worker::run_tasks(); // submitting the first portion of tasks
eng.wait_all();

/**/
cout << "----------\n";
for(int i=0;i<NUM_TASKS;i++)  cout << "arr[" << i << "]=" << arr[i] << endl;;
/**/